### GRASS analysis for Windows 10 version

## Setup

In [ ]:
import geopandas as gpd
import pandas as pd
import altair as alt
import subprocess
import shapely
import folium
import sys
import os
import io
import sys
import shutil

In [ ]:
#print(f"GDAL version: {gdal.__version__}")
import sys
print(f"python version:{sys.version[:6]}")

In [ ]:
# Ask GRASS GIS where its Python packages are.
import subprocess
grass_binary = "/home/anna/dev/grass/grass/bin.x86_64-pc-linux-gnu/grass"
sys.path.append(
    subprocess.check_output([grass_binary, "--config", "python_path"], text=True).strip()
)
# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

In [ ]:
os.chdir("data")
os.getcwd()

In [ ]:
!pwd
!/home/anna/dev/grass/grass/bin.x86_64-pc-linux-gnu/grass -c dtm_clipped.tif -e /home/anna/grassdata/mentoring

In [ ]:
session = gj.init("/home/anna/grassdata/mentoring")

In [ ]:
!g.extension v.transects
!g.extension r.stream.distance
!g.extension r.accumulate

## Analysis

### Channel

In [ ]:
gs.run_command("r.external", input="dtm_clipped.tif", out="dtm")
gs.run_command("r.colors", map="dtm", color="elevation")

In [ ]:
map = gj.Map()
map.d_rast(map='dtm')
map.show()

In [ ]:
gs.run_command("r.stream.extract", elevation="dtm", threshold=500,
                mexp=0.5, stream_length=500, memory=100000, stream_raster="stream_r",
                direction="direction_r", stream_vector="stream_vect")

In [ ]:
map = gj.Map()
map.d_rast(map='direction_r')
map.d_vect(map='stream_vect', color="red", width=2)
map.show()

In [ ]:
gs.run_command("r.accumulate", direction="direction_r", format="auto", accumulation="accum",
               outlet="stream_vect", longest_flow_path="longest_stream")

In [ ]:
map = gj.Map()
map.d_rast(map='accum')
map.d_vect(map='longest_stream', color="blue")
map.show()


In [ ]:
gs.run_command("r.stream.order", stream_rast="stream_r", accumulation="accum", direction="direction_r",
                elevation="dtm", hack="stream_hack", stream_vect="stream_v")

In [ ]:
map = gj.Map()
map.d_rast(map='accum')
map.d_rast(map='stream_hack')
map.show()

In [ ]:
# derive main channel
gs.mapcalc("main_channel = if(stream_hack != 1, null(), 1)")
gs.run_command("r.to.vect", input="main_channel", output="mc_vect", type="line")
# get rid of an artifact during vectorization (removes lines smaller than 1 m)
gs.run_command("v.edit", map="mc_vect", type="line", tool="delete", threshold="-1,0,-1", query="length")
gs.run_command("v.build.polylines", input="mc_vect", output="polylines", cats="first")

In [ ]:
map = gj.Map()
map.d_rast(map='dtm')
map.d_vect(map='polylines')
map.show()

In [ ]:
# get longest channel
lengths = gs.read_command("v.to.db", flags="p", map="polylines", type="line", option="length", columns="aa", separator=",")
df = pd.read_csv(io.StringIO(lengths))
cat = df.iloc[df['length'].idxmax(), 0]
gs.run_command("v.extract", input="polylines", output="plong", cat=cat)

In [ ]:
map = gj.Map()
map.d_rast(map='dtm')
map.d_vect(map='plong')
map.show()

In [ ]:
gs.run_command("v.generalize", input="plong", type="line", method="snakes", threshold=1, output="plong_simp")

### Transects

In [ ]:
gs.run_command("r.stream.distance", stream_rast="stream_r", direction="direction_r", elevation="dtm", difference="hand")

In [ ]:
gs.run_command("v.transects", input="plong_simp", output="transects",
               transect_spacing=10, transect_perpendicular="trend")
gs.run_command("v.to.points", input="transects", out="points", use="vertex")

result = gs.read_command("v.out.ascii", input="points", separator=",")
df = pd.DataFrame([row.rsplit(',',1) for row in result.splitlines()],
                  columns=["coors","cat"])
df = df.groupby('cat')['coors'].apply(','.join).reset_index()
df = df.astype({"cat": int}).sort_values(by=['cat'], ignore_index=True)
df

In [ ]:
html = []
for index, row in df.iterrows():
    profile = gs.read_command("r.profile", input="hand",
                              coordinates=(row["coors"]))
    profile_df = pd.DataFrame([row.split() for row in profile.splitlines()],
                              columns=["distance_(m)", "elevation_(m)"]).apply(pd.to_numeric, errors='coerce')
    line = alt.Chart(profile_df).mark_line().encode(
        alt.X('distance_(m)'),
        alt.Y('elevation_(m):Q',scale=alt.Scale(zero=False))
    ).properties(width=600,height=100)
    test = line.to_dict()
    #vega_lite = folium.VegaLite(line, width='100%',height='100%')
    html.append(test)

In [ ]:
gs.run_command("v.out.ogr", input="transects", output="transects.geojson", format="GeoJSON")
trnscts = gpd.read_file('transects.geojson').to_crs(4326)
trnscts['html'] = html

In [ ]:
gs.run_command("r.geomorphon", forms="forms", elevation="dtm", search=10, skip=3, flat=2.5)

## Visualization

In [ ]:
gs.run_command("v.in.ogr", input="meadow_extent.geojson", output="meadow_extent")
meadow_extent = gs.parse_command("g.region", vector="meadow_extent", flags="ucgl")
loc = (float(meadow_extent["center_long"]), float(meadow_extent["center_lat"]))

In [ ]:
m = folium.Map(location=[loc[1], loc[0]], zoom_start=14, max_zoom=20)
gj.Raster('forms').add_to(m)
gj.Raster('hand').add_to(m) #add HAND
gj.Raster('dtm').add_to(m)
gj.Vector('plong_simp').add_to(m)
gj.Vector('meadow_extent').add_to(m)

fg =folium.FeatureGroup(name='Transects').add_to(m)
for _, s in trnscts.iterrows():
    graph = s['html']
    coords = list(s['geometry'].coords)
    ttip = s['cat']

    geo_data={"type":"Polygon","coordinates":[coords]}
    geo_obj=folium.GeoJson(geo_data, style_function=lambda x:{'color':'gray','weight':4,'dashArray':"2,2"},
                            highlight_function=lambda x:{'color':'yellow','weight':8},
                            tooltip=folium.Tooltip(ttip),
                            popup=folium.Popup().add_child(folium.VegaLite(graph,width="100%")),
                            #popup_keep_highlighted=True, #needs python 3.9?
                            )
    geo_obj.add_child(folium.Popup(max_width=500).add_child(folium.VegaLite(graph)))
    geo_obj.add_to(fg)
m.add_child(folium.LayerControl())
m

In [ ]:
#save folium map as an html object for sharing
m.save('lacey_meadow.html')